In [3]:
import os
import json
import pandas as pd
import numpy as np



In [4]:
# ########################################################
# # DATA SAVE TO CSV
# ########################################################
# import os
# import json
# import pandas as pd

# # Define the root directory
# root_dir = r"data/raw/data_store/new_data"

# # List to store processed data
# all_data = []

# # Walk through all folders and files
# for subdir, _, files in os.walk(root_dir):
#     for file in files:
#         if file.endswith(".json"):
#             file_path = os.path.join(subdir, file)

#             # Read JSON file
#             with open(file_path, "r", encoding="utf-8") as f:
#                 data = json.load(f)

#                 # Extract main fields and flatten 'tags', add source folder and file name
#                 for item in data:
#                     flat_item = {
#                         "lat": item.get("lat"),
#                         "lon": item.get("lon"),
#                         # Flatten tags into separate columns
#                         **item.get("tags", {}),
#                         # Get only the folder name
#                         "source_folder": os.path.basename(subdir),
#                         # Get only the file name
#                         "file_name": os.path.basename(file)
#                     }
#                     all_data.append(flat_item)

# # Convert to DataFrame
# df = pd.DataFrame(all_data)

# # Write DataFrame to CSV
# csv_output_path = r"data/raw/csv/data.csv"
# df.to_csv(csv_output_path, index=False)

# # Display the first few rows to verify
# print(df.head())

In [5]:
# Load your CSV data
df = pd.read_csv(r"data/raw/csv/data.csv")
df.head()

,lat,lon,addr:city,addr:district,description,opening_hours,phone,branch,amenity,mobile,...,cuisine,name,name:en,name:uk,name:de,name:tr,name:ru,name:ar,source_folder,file_name
0,41.033610,28.978564,İstanbul,NaN,NaN,NaN,NaN,NaN,bar,NaN,...,NaN,Urban Cafe,NaN,NaN,NaN,NaN,NaN,إيربان,amenity,bar.json
1,41.028784,28.974217,İstanbul,NaN,NaN,NaN,NaN,NaN,bar,NaN,...,NaN,Badehane,NaN,NaN,NaN,NaN,NaN,بادي هاني,amenity,bar.json
2,41.032744,28.976723,İstanbul,Beyoğlu,NaN,NaN,+90 533 691 0 360,NaN,bar,NaN,...,NaN,360 Istanbul,NaN,NaN,NaN,NaN,NaN,اسطنبول 360,amenity,bar.json
3,41.033616,28.984524,İstanbul,NaN,NaN,NaN,NaN,NaN,bar,NaN,...,NaN,Roxy Bar,NaN,NaN,NaN,NaN,NaN,بار روكسي,amenity,bar.json
4,41.030045,28.974339,İstanbul,NaN,NaN,NaN,NaN,NaN,bar,NaN,...,NaN,Kino Garden,NaN,NaN,NaN,NaN,NaN,NaN,amenity,bar.json


In [6]:
import pandas as pd
import re

# Load your CSV data
df = pd.read_csv(r"data/raw/csv/data.csv")

# List of known regex patterns
known_patterns: list[str] = [
    r"^\+90 \d{3} \d{3} \d \d{3}$",  # +90 533 691 0 360
    r"^\d{4} \d{3} \d{2} \d{2}$",    # 0212 257 12 55
    r"^\+90 \(\d{3}\) \d{3} \d{4}$",  # +90 (212) 275 8888
    r"^\(\d{4}\) \d{3} \d{2} \d{2}$",  # (0216) 348 48 18
    r"^\d{3} \d{4}$",                 # 244 1656
    r"^\d{11}$",                     # 02163995361
    r"^\+90 \d{3} \d{3} \d{2} \d{2}$",  # +90 212 252 53 46
    r"^\+90 \d{3} \d{3} \d{4}$",               # +90 212 310 2525
    r"^0\d{3}\d{10}$",                         # 0905493261112
    r"^0 \d{3} \d{3} \d{4}$",                 # 0 216 360 9090
    r"^\d{4} \d{3} \d{4}$",                   # 0216 318 1016
    r"^\d{7}$",                                # 5162420
    r"^\+90\(\d{3}\)\d{7}$",                  # +90(216)6613131
    r"^\d{4} \d{7}$",                         # 0212 2584520
    r"^\+90 \d{3} \d{7}$",                    # +90 216 418 2468
    r"^\+90 \d{1} \d{3}$",                     # +90 444 8 845
    r"^\+90 \(\d{3}\) \d{3} \d{2} \d{2}$",    # +90 (212) 299 84 87
    r"^90 \d{3} \d{3} \d{4}$",                # 90 216 504 10 65
    r"^00\d{3}\d{10}$",                       # 00905445437556
    r"^\d{10}$",                               # 5555532072
    r"^\+90\d{10}$",                           # +90263370793
    r"^\d{13}$",                               # 0905493261112
    r"^\d{2} \d{3} \d{3} \d{2} \d{2}$",       # 90 216 504 10 65
    r"^\+\d{11}$",                             # +90263370793
    r"^\+\d{2} \d{7}$",                       # +90 4446661
    r"^\+\d{2} \d{10}$",                      # +90 8507550774
    r"^\+\d{3} \d{3} \d{2} \d{2}$",           # +535 705 09 27
    r"^\d{3} \d{3} \d{2} \d{2}$",             # 212 380 03 42
    r"^\d{1} \(\d{3}\) \d{3} \d{2} \d{2}$",   # 0 (212) 871 24 23
    r"^\d{3} \d{1} \d{3}$",                   # 444 5 464
    r"^\+\d{12};\+\d{12}$",                   # +905434074077;+905428032521
    r"^\d{10}-\d{11}$",                       # 0214051124-05556275555
    r"^\d{3} \d{2} \d{2}$",                   # 664 98 88
    r"^\d{1} \d{3} \d{3} \d{2} \d{2}$",       # 0 216 467 77 55
    r"^\+\d{9}$",                             # +904443322
    r"^\+\d{2} \d{4} \d{3} \d{2} \d{2}$",     # +90 0540 728 28 18
    r"^\+\d{2} \d{3} \d{4}$",                 # +90 444 6262
    r"^\+\d{2} \d{3} \d{3} \d{2} \d{2};\+\d{2} \d{3} \d{3} \d{2} \d{2}$",
    r"^HOTEL \d{3} \d{2} \d{2}$",             # HOTEL 528 11 46
    r"^\+\d{2} \d{3} \d{7};\+\d{2} \d{3} \d{7}$",
    r"^\+\d{5} \d{3} \d{4}$",                 # +90212 243 8190
    r"^\+\d{2} \(\d{4}\) \d{3} \d{4}$",       # +90 (0212) 271 0084
    r"^\+\d{2} \d{6} \d{2} \d{2}$",           # +90 212249 03 93
    r"^\+\d{2} \d{4} \d{3} \d{4}$",           # +90 0212 251 7991
    r"^\(\d{4}\)\d{7}$",                      # (0212)2935656
    r"^\+\d{2} \d{11}$",                      # +90 05322961178
    r"^\+\d{2} \d{1} \d{3} \d{3} \d{2} \d{2}$",  # +90 0 212 353 05 12
    r"^\d{12}$",                              # 021253224429
    r"^Can Pide Ve Lahmacun Fırını$",         # Can Pide Ve Lahmacun Fırını
    r"^Itfaiye Caddesi$",                     # Itfaiye Caddesi
    r"^\d{8}$",                               # 05889005
    r"^\d{4}\) \d{3} \d{2} \d{2}$",           # 0212) 585 99 59
    r"^\+\d{2} \d{3} \d{7}/\d{4}$",           # +90 212 3156000/8426
    r"^\d{4} \d{3}-\d{2}-\d{2}$",             # 0212 234-24-34
    r"^\+\d{15}$",                            # +902128502010939
    r"^\+\d{2} \d{3} \d{5}-\d{2}$",           # +90 216 34463-69
    r"^\+\d{2} \(\d{3}\) \d{3} \d{2}$",       # +90 (212) 262 62
    r"^\d{1} \d{3} \d{7}$",                   # 0 216 4822425
    r"^\+\d{2}-\d{3}-\d{7}$",                 # +90-216-7712141
    r"^\+\d{2} \d{4} \d{3} \d{2} \d{2};\+\d{2} \d{4} \d{3} \d{2} \d{2};\+\d{2} \d{3} \d{3} \d{2} \d{2}$",
    r"^\+\d{2} \(\d{1}\) \d{3} \d{3} \d{2} \d{2}$",  # +90 (0) 212 244 44 99
    r"^\+\d{2} \(\d{4}\) \d{3} \d{2} \d{2}$",   # +90 (0212) 561 27 27
    r"^\+\d{5} \d{3} \d{2} \d{2}$",           # +90531 217 18 58
    r"^\+\d{2}  \d{3} \d{3} \d{4}$",          # +90 	216 391 2119
    r"^\+\d{12}and\+\d{12}$",                 # +905305516676 and +902163635836
    r"^\d{4} \d{3} \d{4}‎$",                 # 0212 293 6968‎
    r"^\(\+\d{2}-\d{3}-\d{4}\) \(\d{4}-\d{7}\)$",
    r"^\d{5}-\d{3} \d{4}$",                   # 0532-345 5424
    r"^\+90 \d{2} \d{5} \d{7}$",              # +90 537 224 1005
    r"^\d{10}.\d{5}.\d{6}$",                  # 5351264884.74130.21735
    r"^\+\d{2} \d{3} \d{3} \d{4}$",           # +90 216 391 2119
    r"^\+\d{12}$",                           # +902163635836
    r"^\(\+\d{2}-\d{3}\) \d{3}-\d{4}$",       # (+90-212) 251-2858
    r"^\d{11} \d{11}$",                       # 05054777961 05052389122
    r"^\+ \d{2} \d{3} \d{3} \d{4}$",          # + 90 212 366 4608
    r"^\+\d{2} \d{3} \d{3}-\d{4}$",           # +90 531 942-0619
    r"^\+\d{2} \d{3} \d{2} \d{2}$",           # +49 216 566 96 54
    r"^\d{4} \d{3} \d{2}$",                   # 0212 576 80
    r"^\+\d{10} \d{2}$",                      # +9085033399 88
    r"^\d{4} \d{3} \d{1} \d{3} - \d{4} \d{3} \d{1} \d{3}$",
    r"^ا$",                                   # ا
    r"^\d{1}\.sk$",                           # 1.sk
    r"^\+\d{12}bb$",                          # +902163131484bb
    r"^\+\d{2} \d{3} \d{3}  \d{4}$",           # +90 216 384  9827
    r"^\d{4} \d{3} \d{2} \d{2} - \d{3} \d{2} \d{2}$",
    r"^\d{9}$",                               # 055551413
    r"^\+\d{5} \d{7}$",                        # +90212 8019319
    r"^\+\d{10}$",                            # +2127779701
    r"^\d{1} \(\d{3}\) \d{3} \d{4}$",          # 0 (212) 481 0029
    r"^\+\d{13}$",                            # +9002166300630
    r"^‭\d{1} \(\d{3}\) \d{3} \d{2} \d{2}‬$",  # ‭0 (541) 901 29 94‬
    r"^\+\d{2}\(\d{3}\)\d{3}-\d{2}-\d{2}$",    # +90(216)380-83-80
    r"^\d{2}\.Blv Cd$",                        # 58.Blv Cd
    r"^\d{3} \d{2} \d{2} - \d{3} \d{2} \d{2}$",  # 558 62 73 - 510 32 33
    r"^\d{3}-\d{3}-\d{5}$",                    # 053-361-88962
    r"^\(\d{4}\) \d{3} \d{4}$",                # (0216) 541 4161
    r"^\d{1}\(\d{3}\) \d{3} \d{2} \d{2}$",      # 0(212) 590 97 87
    r"^\+\d{2} \d{3} \d{2} \d{2} \d{3}$",       # +90 212 87 67 655
    r"^https://www\.barker\.com\.tr/$",         # https://www.barker.com.tr/
    r"^mo-su \d{2}:\d{2}-\d{2}:\d{2}$",         # mo-su 08:00-22:00
    r"^\d{11} - \d{11}$",                       # 02122687507 - 05326881675
    r"^\d{2}:\d{2}-\d{2}:\d{2}$",               # 09:00-23:00
    r"^\+\d{2} \d{4} \d{3} \d{3}$",             # +61 1800 761 192
    r"^\+\d{2} \d{2}$",                         # +90 53
    r"^\d{4} \d{3} \d{5}$",                     # 0553 088 22299
    r"^\+\d{2} \d{3} \d{3} \d{4}$",             # For +90 216 391 2119
    # For +905305516676 and +902163635836
    r"^\+\d{12} and \+\d{12}$",
    r"^\+\d{2} \d{3} \d{3} \d{2} \d{2}$",        # For +49 216 566 96 54
    r"^\d{11} \d{11} \d{11}$",
    r"^\+\d{12}(?:\s*(?:and|\n)\s*\+\d{12})+$",
    r"^\d{11}   \d{11}   \d{11}$"
]


def normalize_phone_number(phone_number):
    """Normalize phone numbers if they match one of the known patterns.
       Otherwise, return the original phone number.
    """
    if not isinstance(phone_number, str) or pd.isna(phone_number):
        return phone_number

    # Normalize whitespace: remove extra spaces and tabs
    phone_number = " ".join(phone_number.split())

    # Check against the known regex patterns
    valid = any(re.fullmatch(pattern, phone_number)
                for pattern in known_patterns)
    if not valid:
        return phone_number

    # Clean the phone number: keep digits (and leading '+' if present)
    if phone_number.startswith('+'):
        cleaned = '+' + re.sub(r'\D', '', phone_number[1:])
    else:
        cleaned = re.sub(r'\D', '', phone_number)

    # Normalize based on the structure of the cleaned number
    if cleaned.startswith('90') and len(cleaned) == 12:
        return f"+90 {cleaned[2:5]} {cleaned[5:8]} {cleaned[8:]}"
    elif cleaned.startswith('+90') and len(cleaned) == 13:
        return f"+90 {cleaned[3:6]} {cleaned[6:9]} {cleaned[9:]}"
    elif cleaned.startswith('0') and len(cleaned) == 11:
        return f"0 {cleaned[1:4]} {cleaned[4:7]} {cleaned[7:9]} {cleaned[9:]}"
    elif len(cleaned) == 7:
        return f"{cleaned[:3]} {cleaned[3:]}"
    elif len(cleaned) == 10 and cleaned.startswith('0'):
        return f"0 {cleaned[1:4]} {cleaned[4:7]} {cleaned[7:9]} {cleaned[9:]}"
    elif len(cleaned) == 10:
        return f"{cleaned[:3]} {cleaned[3:6]} {cleaned[6:]}"
    elif ';' in phone_number:
        # Normalize each part in case of multiple numbers separated by ';'
        parts = [normalize_phone_number(p.strip())
                 for p in phone_number.split(';')]
        return "; ".join(parts)
    else:
        return cleaned


# Apply normalization and generate a phone format representation
df['normalized_phone'] = df['phone'].apply(normalize_phone_number)
df['phone_format'] = df['normalized_phone'].apply(
    lambda s: re.sub(r'\d', 'X', s) if isinstance(s, str) else s)

# The valid simplified formats as observed previously
expected_formats = [
    "X XXX XXX XX XX",   # e.g., 0 216 467 77 55
    "+XX XXX XXX XXXX",  # e.g., +90 212 310 2525
    "XXX XXXX",          # e.g., 516 2420
    "XXX XXX XXXX",      # e.g., 212 380 0342
    "+XXXXXXXXXXXXX"     # e.g., a cleaned number that didn’t match other conditions
]

# Create a new column for the format category based on phone_format


def get_format_category(phone_format):
    if not isinstance(phone_format, str):
        return None
    for fmt in expected_formats:
        if phone_format == fmt:
            return fmt
    return "Other"


df['format_category'] = df['phone_format'].apply(get_format_category)

# Function to determine if a phone number is valid based on known patterns.


def is_valid_phone(phone_number):
    if not isinstance(phone_number, str) or phone_number.strip() == "":
        return False
    phone_number = " ".join(phone_number.split())
    return any(re.fullmatch(pattern, phone_number) for pattern in known_patterns)


# Create a new column 'is_valid' in the DataFrame
df['is_valid'] = df['phone'].apply(is_valid_phone)

# Now, the original DataFrame 'df' contains the following new columns:
#   - 'normalized_phone': the normalized phone number based on the known patterns.
#   - 'phone_format': a simplified format (digits replaced with X) for easier comparison.
#   - 'format_category': which expected format the phone matches (or "Other" if none).
#   - 'is_valid': True if the original phone number matched one of the known patterns, otherwise False.

# You can now proceed with further analysis directly on 'df'
df.head()

,lat,lon,addr:city,addr:district,description,opening_hours,phone,branch,amenity,mobile,...,name:de,name:tr,name:ru,name:ar,source_folder,file_name,normalized_phone,phone_format,format_category,is_valid
0,41.033610,28.978564,İstanbul,NaN,NaN,NaN,NaN,NaN,bar,NaN,...,NaN,NaN,NaN,إيربان,amenity,bar.json,NaN,NaN,None,False
1,41.028784,28.974217,İstanbul,NaN,NaN,NaN,NaN,NaN,bar,NaN,...,NaN,NaN,NaN,بادي هاني,amenity,bar.json,NaN,NaN,None,False
2,41.032744,28.976723,İstanbul,Beyoğlu,NaN,NaN,+90 533 691 0 360,NaN,bar,NaN,...,NaN,NaN,NaN,اسطنبول 360,amenity,bar.json,+90 533 691 0360,+XX XXX XXX XXXX,+XX XXX XXX XXXX,True
3,41.033616,28.984524,İstanbul,NaN,NaN,NaN,NaN,NaN,bar,NaN,...,NaN,NaN,NaN,بار روكسي,amenity,bar.json,NaN,NaN,None,False
4,41.030045,28.974339,İstanbul,NaN,NaN,NaN,NaN,NaN,bar,NaN,...,NaN,NaN,NaN,NaN,amenity,bar.json,NaN,NaN,None,False


In [7]:
import pandas as pd
import numpy as np
import re

# Assuming you have the DataFrame `df` already loaded

# Step 1: Clean phone numbers (Remove +90 and leading 0)
df['cleaned_phone'] = df['normalized_phone'].astype(
    str).str.replace(r'^\+90|^0', '', regex=True)

# Step 2: Remove special characters (like ;, +, etc.)
df['cleaned_phone'] = df['cleaned_phone'].str.replace(
    r'[^0-9]', '', regex=True)

# Step 3: Replace digits with 'x' and clean up non-useful entries (nan, empty strings, etc.)
df['format'] = df['cleaned_phone'].apply(
    lambda x: re.sub(r'\d', 'x', x) if pd.notna(x) else '')

# Remove empty values and invalid formats
df_cleaned = df[df['format'] != '']

# Get unique formats
unique_formats = df_cleaned['format'].unique()
print("Unique formats:", unique_formats)

# Step 4: Normalize to a standard format
df['normalized'] = df['cleaned_phone'].str.replace(
    r'\D', '', regex=True)  # Remove non-digits

# Step 5: Handle shorter numbers like 5238895 -> xxx xxxx


def normalize_short_number(number):
    if len(number) == 7:  # Short numbers like 5238895
        return f"{number[:3]} {number[3:]}"
    return number

# Step 6: Handle long numbers (e.g., 5434074077905428032521) and keep only the first part


def normalize_long_number(number):
    # Split if length exceeds 10 digits and only keep the first part
    if len(number) > 10:
        return number[:10]
    return number

# Step 7: Check if number has fewer than 4 digits and set to NaN if so


def handle_invalid_number(number):
    # If the number has fewer than 4 digits, return NaN
    if len(number) < 4:
        return np.nan
    return number


# Normalize phone numbers
df['normalized'] = df['normalized'].apply(normalize_long_number)
df['normalized'] = df['normalized'].apply(normalize_short_number)

# Apply invalid number handling (less than 4 digits becomes NaN)
df['normalized'] = df['normalized'].apply(handle_invalid_number)

# Clean up empty or invalid entries
df['normalized'] = df['normalized'].replace("", np.nan)

# Display cleaned and normalized phone numbers
filtered_data = df[df['normalized'].notna()]['normalized']

# Save to a text file
filtered_data.to_csv('output1.txt', index=False, header=False)

# Print out the cleaned data to inspect
print(filtered_data.head())

Unique formats: ['xxxxxxxxxx' 'xxxxxxxxxxxx' 'xxxxxxx' 'xxxxxxxxxxxxx' 'xxxxxxxxx'
 'xxxxxxxxxxxxxxxxxxxxxx' 'xxxxxxxxxxxxxxxxxxxx' 'xxxxxxxxxxx'
 'xxxxxxxxxxxxxx' 'xxxxxxxx' 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
 'xxxxxxxxxxxxxxxxxxxxx' 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx' 'x'
 'xxxxxxxxxxxxxxxxx' 'xx']
2     5336910360
11    2123374500
21    2122571255
24    2122273369
70    5325934542
Name: normalized, dtype: object


In [8]:
df.head()

,lat,lon,addr:city,addr:district,description,opening_hours,phone,branch,amenity,mobile,...,name:ar,source_folder,file_name,normalized_phone,phone_format,format_category,is_valid,cleaned_phone,format,normalized
0,41.033610,28.978564,İstanbul,NaN,NaN,NaN,NaN,NaN,bar,NaN,...,إيربان,amenity,bar.json,NaN,NaN,None,False,,,NaN
1,41.028784,28.974217,İstanbul,NaN,NaN,NaN,NaN,NaN,bar,NaN,...,بادي هاني,amenity,bar.json,NaN,NaN,None,False,,,NaN
2,41.032744,28.976723,İstanbul,Beyoğlu,NaN,NaN,+90 533 691 0 360,NaN,bar,NaN,...,اسطنبول 360,amenity,bar.json,+90 533 691 0360,+XX XXX XXX XXXX,+XX XXX XXX XXXX,True,5336910360,xxxxxxxxxx,5336910360
3,41.033616,28.984524,İstanbul,NaN,NaN,NaN,NaN,NaN,bar,NaN,...,بار روكسي,amenity,bar.json,NaN,NaN,None,False,,,NaN
4,41.030045,28.974339,İstanbul,NaN,NaN,NaN,NaN,NaN,bar,NaN,...,NaN,amenity,bar.json,NaN,NaN,None,False,,,NaN


In [9]:
# Drop specified columns and rename 'normalized' to 'phone_number'
df = df.drop(columns=['phone', 'normalized_phone', 'phone_format',
             'format_category', 'cleaned_phone', 'format', 'mobile', "is_valid"])

# Rename 'normalized' to 'phone_number'
df = df.rename(columns={'normalized': 'phone_number'})

# Display the first few rows of the DataFrame
df.head()

,lat,lon,addr:city,addr:district,description,opening_hours,branch,amenity,addr:street,cuisine,name,name:en,name:uk,name:de,name:tr,name:ru,name:ar,source_folder,file_name,phone_number
0,41.033610,28.978564,İstanbul,NaN,NaN,NaN,NaN,bar,"Kartal Sokak 6/A, Istanbul",NaN,Urban Cafe,NaN,NaN,NaN,NaN,NaN,إيربان,amenity,bar.json,NaN
1,41.028784,28.974217,İstanbul,NaN,NaN,NaN,NaN,bar,General Yazgan Sokak No: 5 Tünel,NaN,Badehane,NaN,NaN,NaN,NaN,NaN,بادي هاني,amenity,bar.json,NaN
2,41.032744,28.976723,İstanbul,Beyoğlu,NaN,NaN,NaN,bar,İstiklal Caddesi,NaN,360 Istanbul,NaN,NaN,NaN,NaN,NaN,اسطنبول 360,amenity,bar.json,5336910360
3,41.033616,28.984524,İstanbul,NaN,NaN,NaN,NaN,bar,NaN,NaN,Roxy Bar,NaN,NaN,NaN,NaN,NaN,بار روكسي,amenity,bar.json,NaN
4,41.030045,28.974339,İstanbul,NaN,NaN,NaN,NaN,bar,NaN,NaN,Kino Garden,NaN,NaN,NaN,NaN,NaN,NaN,amenity,bar.json,NaN


In [10]:
import numpy as np

# Ensure phone_number column is treated as strings and handle NaN values
df['phone_number'] = df['phone_number'].fillna('').astype(str)

# Set phone numbers that start with '0' to NaN
df.loc[df['phone_number'].str.startswith('0'), 'phone_number'] = np.nan

In [11]:
df['phone_number'].str.startswith('0').value_counts()

phone_number
False    47169
Name: count, dtype: int64

In [12]:
def add_country_code(phone_number):
    # Check if the phone number is None, empty string, or non-string
    if not phone_number or not isinstance(phone_number, str):
        return phone_number  # Return as is if it's None, empty string, or non-string

    # Check if phone number is in the 'xxx xxxx' format
    if re.match(r'^\d{3} \d{4}$', phone_number):
        return phone_number
    else:
        return '+90' + phone_number


# Apply the function to the 'phone_number' column
df['phone_number'] = df['phone_number'].apply(add_country_code)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47194 entries, 0 to 47193
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   lat            47194 non-null  float64
 1   lon            47194 non-null  float64
 2   addr:city      47194 non-null  object 
 3   addr:district  16830 non-null  object 
 4   description    281 non-null    object 
 5   opening_hours  5045 non-null   object 
 6   branch         288 non-null    object 
 7   amenity        47194 non-null  object 
 8   addr:street    17722 non-null  object 
 9   cuisine        2576 non-null   object 
 10  name           35041 non-null  object 
 11  name:en        1247 non-null   object 
 12  name:uk        9 non-null      object 
 13  name:de        34 non-null     object 
 14  name:tr        940 non-null    object 
 15  name:ru        194 non-null    object 
 16  name:ar        13513 non-null  object 
 17  source_folder  47194 non-null  object 
 18  file_n

In [14]:
# Remove the '.json' from the values in the 'Source_file' column
df['file_name'] = df['file_name'].str.replace(
    '.json', '', regex=False)

In [15]:
# Create a dictionary with the old column names as keys and the new names as values
new_column_names = {
    'lat': 'latitude',
    'lon': 'longitude',
    'addr:city': 'city',
    'addr:district': 'district',
    'description': 'description',
    'opening_hours': 'opening_hours',
    'branch': 'branch',
    'amenity': 'amenity',
    'addr:street': 'street',
    'cuisine': 'cuisine',
    'name': 'name',
    'name:en': 'name_en',
    'name:uk': 'name_uk',
    'name:de': 'name_de',
    'name:tr': 'name_tr',
    'name:ru': 'name_ru',
    'name:ar': 'name_ar',
    'source_folder': 'category',
    # Changed from 'source_file' to 'file_name' to match your original column
    'file_name': 'subcategory',
    'phone_number': 'phone_number'
}

# Rename columns in the DataFrame
df = df.rename(columns=new_column_names)

# Reorder the columns in the desired order
new_order = [
    'latitude', 'longitude', 'city', 'district', 'street', 'branch', 'phone_number',
    'description', 'opening_hours', 'amenity', 'cuisine', 'name', 'name_en', 'name_uk',
    'name_de', 'name_tr', 'name_ru', 'name_ar', 'category', 'subcategory',
]

# Reorder the columns
df = df[new_order]

# Check the updated column order
print(df.head())

    latitude  longitude      city district                            street  \
0  41.033610  28.978564  İstanbul      NaN        Kartal Sokak 6/A, Istanbul   
1  41.028784  28.974217  İstanbul      NaN  General Yazgan Sokak No: 5 Tünel   
2  41.032744  28.976723  İstanbul  Beyoğlu                  İstiklal Caddesi   
3  41.033616  28.984524  İstanbul      NaN                               NaN   
4  41.030045  28.974339  İstanbul      NaN                               NaN   

  branch   phone_number description opening_hours amenity cuisine  \
0    NaN                        NaN           NaN     bar     NaN   
1    NaN                        NaN           NaN     bar     NaN   
2    NaN  +905336910360         NaN           NaN     bar     NaN   
3    NaN                        NaN           NaN     bar     NaN   
4    NaN                        NaN           NaN     bar     NaN   

           name name_en name_uk name_de name_tr name_ru      name_ar category  \
0    Urban Cafe     NaN

In [16]:
# Get unique values from 'category' and 'subcategory'
unique_categories = df['category'].unique()
unique_subcategories = df['subcategory'].unique()

# Create DataFrames for each unique column
category_df = pd.DataFrame(unique_categories, columns=['category'])
subcategory_df = pd.DataFrame(unique_subcategories, columns=['subcategory'])

# Save each DataFrame to CSV
category_df.to_csv('category.csv', index=False)
subcategory_df.to_csv('subcategory.csv', index=False)

In [17]:
# Write DataFrame to CSV
csv_output_path = r"data/processed/processed_data_phone_cleaned.csv"
df.to_csv(csv_output_path, index=False)